In [ ]:
%reload_ext autoreload
%autoreload 2

import requests
from pprint import pprint
from datetime import datetime
from math import ceil
import matplotlib.pyplot as plt
import asyncio
import pandas as pd
import decimal
import warnings
warnings.filterwarnings('ignore')
from copy import deepcopy as dc
import time
import os
import json
import sys
# --------------------------------------------------------------
from bitget.consts import CONTRACT_WS_URL
from bitget.ws.bitget_ws_client import BitgetWsClient, SubscribeReq
from bitget.mix.order_api import OrderApi
# --------------------------------------------------------------
import plotly.graph_objs as go
import plotly.express as px
# from mplfinance import candlestick2_ohlc
import pylab as pl
from IPython import display as display_graph
%matplotlib inline
# %matplotlib widget
# --------------------------------------------------------------
from indicators import ichimoku_and_ema, plot_close_indicators
from strategies import strategy



# # Permanently changes the pandas settings
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

plt.rcParams["figure.figsize"] = (20, 16)
            
timeFrames = {
    60: '1m',
    180: '3m',
    300: '5m',
    900: '15m',
    1800: '30m',
    3600: '1H',
    7200: '2H',
    14400: '4H',
    21600: '6H' 
}      

# --------------------------------------------------------------

def placeBuy(size):
    global order_api
    while True:
        try:
            order_api.place_order(symbol = 'SBTCSUSDT_SUMCBL', marginCoin = 'SUSDT', size = size, side = 'close_short', orderType = 'market')
        except:
            break
    order_api.place_order(symbol = 'SBTCSUSDT_SUMCBL', marginCoin = 'SUSDT', size = size, side = 'open_long', orderType = 'market')

def placeSell(size):
    global order_api
    while True:
        try:
            order_api.place_order(symbol = 'SBTCSUSDT_SUMCBL', marginCoin = 'SUSDT', size = size, side = 'close_long', orderType = 'market')
        except:
            break
    order_api.place_order(symbol = 'SBTCSUSDT_SUMCBL', marginCoin = 'SUSDT', size = size, side = 'open_short', orderType = 'market')

def ws_app():

    def handle(message):
        global currentPrice
        global nextTimeBoundary
        global timeframe
        global firstCandle
        global minPrice
        global maxPrice
        global openn
        global high
        global low
        global close
        global ws_df
        global plot
        global ema_lags
        global plot
        global refresh_plot
        global size
        
        json_message = json.loads(message)
        prevPrice = currentPrice
        currentPrice = float(json_message['data'][0]['markPrice']) 
        currentTime = int(json_message['data'][0]['systemTime'])/1000
        if currentTime > nextTimeBoundary:
            nextTimeBoundary += timeframe

            if firstCandle:
                firstCandle = False
                low = currentPrice
                high = currentPrice
                openn = currentPrice
            else:
                low = minPrice 
                high = maxPrice
                close = prevPrice
                
                candleStart = nextTimeBoundary - timeframe
                ws_df = ws_df.append({'Date/Time': datetime.fromtimestamp(candleStart),
                                'Timestamp': int(candleStart),
                                'Open': openn,
                                'High': high,
                                'Close': close,
                                'Low': low},
                                ignore_index=True)

                min_ichi = 60
                ichi = True
                if not ichi:
                    min_ichi = max(ema_lags)
                else:
                    min_ichi = max(min_ichi, max(ema_lags))


                if len(ws_df) > min_ichi:
                    df_ind = ichimoku_and_ema(ws_df, timeframe, ema_lags)
                    strategy(df_ind, ema_lags, ichi)
                    if df_ind.iloc[len(ws_df)-1]['Buy'] == True: 
                        placeBuy(size)
                        print("BUY @ {}".format(df_ind.iloc[len(ws_df)-1]['Close']))
                        
                    elif df_ind.iloc[len(ws_df)-1]['Sell'] == True: 
                        placeSell(size)
                        print("SELL @ {}".format(df_ind.iloc[len(ws_df)-1]['Close']))
                        

                    if plot:
                        plot_close_indicators("BTC_USDT", df_ind, refresh_plot, ema_lags)
                    else:
                        print(len(ws_df)-1)
                else:
                    print("{}/{} candles so far\n".format(len(ws_df), min_ichi))
                
                openn = currentPrice
                minPrice = currentPrice
                maxPrice = currentPrice

                
        else:
            minPrice = currentPrice if currentPrice < minPrice else minPrice
            maxPrice = currentPrice if currentPrice > maxPrice else maxPrice

    def handle_error(message):
        print("Handle Error:" + message)

    client = BitgetWsClient(CONTRACT_WS_URL, need_login=False).error_listener(handle_error).build()

    channels = [SubscribeReq("mc","ticker", "BTCUSDT")]
    client.subscribe(channels, handle)

    while True:
        pass

def rest_app(timeframe, ema_lags, size):

    global timeframes
    global plot
    global refresh_plot

    def appendDataFromURL(rest_df, start, end, timeframeStr):
        api_url = "https://api.bitget.com/api/mix/v1/market/candles?symbol=BTCUSDT_UMCBL&granularity={}&startTime={}&endTime={}".format(timeframeStr, 1000*start, 1000*end)

        response = requests.get(api_url)

        for candle in response.json():
            # print("{}: Open = {}\t Highest = {}\t Lowest = {}\t Close = {}\t BTC Volume = {}\t USDT Volume = {}".format(datetime.fromtimestamp(int(candle[0])/1000), *[float(value) for value in candle[1:]]))
            rest_df = rest_df.append({'Date/Time': datetime.fromtimestamp(int(candle[0])/1000),
                            'Timestamp': int(int(candle[0])/1000),
                            'Open': float(candle[1]),
                            'High': float(candle[2]),
                            'Close': float(candle[3]),
                            'Low': float(candle[4])}, 
                            ignore_index = True)
        return rest_df






    timeframeStr = timeFrames[timeframe]

    n = 200
    chunkSize = 100

    inPosition = False

    n = int(chunkSize*ceil(n/chunkSize))

    rest_df = pd.DataFrame(columns=('Date/Time', "Timestamp", 'Open', 'High', 'Close', 'Low'))#, 'Tenkan Sen', 'Kijun Sen', 'Senkou Span A', 'Senkou Span B', 'Chikou Span'))

    currentTimestamp = datetime.now().timestamp()

    #-------------------------------------------------------------
    windows = n//chunkSize
    for i in range(windows-1, -1, -1):
        delay = i*chunkSize*timeframe
        start = int(currentTimestamp - currentTimestamp%timeframe - ((chunkSize)*timeframe) - delay)
        end = int(currentTimestamp - currentTimestamp%timeframe - delay)
        rest_df = appendDataFromURL(rest_df, start, end, timeframeStr)
        df_ind = ichimoku_and_ema(rest_df, timeframe, ema_lags)
        strategy(df_ind, ema_lags)
        if plot:
            plot_close_indicators("BTC_USDT", df_ind, refresh_plot, ema_lags)
        else:
            display(df_ind[len(rest_df)-1])

    print("Historical data shown, now live data")

    #-------------------------------------------------------------
    while True:
        currentTimestamp = datetime.now().timestamp()
        timeLeft = timeframe - currentTimestamp%timeframe + 1
        start = int(currentTimestamp - currentTimestamp%timeframe)# - timeframe)
        end = int(currentTimestamp - currentTimestamp%timeframe + timeframe)
        time.sleep(timeLeft)

        rest_df = appendDataFromURL(rest_df, start, end, timeframeStr)
        
        df_ind = ichimoku_and_ema(rest_df, timeframe, ema_lags)
        strategy(df_ind, ema_lags)

        if plot:
            plot_close_indicators("BTC_USDT", df_ind, refresh_plot, ema_lags)
        display(df_ind.iloc[len(rest_df)-1])
                
        if df_ind.iloc[len(rest_df)-1]['Buy'] == True: # and not inPosition:
            print("BUY @ {}".format(df_ind.iloc[len(rest_df)-1]['Close']))
            placeBuy(size)
        elif df_ind.iloc[len(rest_df)-1]['Sell'] == True: # and inPosition:
            print("SELL @ {}".format(df_ind.iloc[len(rest_df)-1]['Close']))
            placeSell(size)

def run(timeframe, ema_lags, size):
    global nextTimeBoundary
    currentTimestamp = datetime.now().timestamp()
    nextTimeBoundary = int(currentTimestamp - currentTimestamp%timeframe + timeframe)
    if timeframe in timeFrames.keys():
        rest_app(timeframe, ema_lags, size)
    else:
        ws_app()
          

currentPrice = 0
firstCandle = True
minPrice = float('inf')
maxPrice = 0
openn = 0
high = 0
low = 0
close = 0
nextTimeBoundary = -1
    
ws_df = pd.DataFrame(columns=('Date/Time', 'Timestamp', 'Open', 'High', 'Close', 'Low'))

api_key = "bg_020a871b5887d07f4864b96891ad4c69"
secret_key = "e5f1db49382bba6e3a69c4e45749e9407215c1e627e167fff4b1262756f51ca8"
passphrase = "couscous"

order_api = OrderApi(api_key, secret_key, passphrase)




In [ ]:
#######################################################################################
timeframe = 1 #int(input("Enter timeframe in seconds: ")) # in seconds

ema_lags = (25, 50)

size = 0.01 # BTC per buy/sell

plot = True
refresh_plot = True
#######################################################################################

run(timeframe, ema_lags, size)
 